In [1]:
!pip install accelerate transformers einops langchain xformers bitsandbytes sentence_transformers -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 6.8 MB/s eta 0:00:00


In [2]:
!pip install gradio faiss-gpu  PyMuPDF gdown pypdf -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 11.4 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install langchain-community -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.3 MB/s eta 0:00:00


In [5]:
import gradio as gr
import torch
from transformers import pipeline, AutoTokenizer,AutoConfig,AutoModelForCausalLM, BitsAndBytesConfig, StoppingCriteria, StoppingCriteriaList
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain

In [6]:
# Initialize the Hugging Face Pipeline
model_id = 'NousResearch/Llama-2-7b-chat-hf'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)
model_config = AutoConfig.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto'
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [7]:
model.eval()
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [8]:
stop_list = ['\nHuman:', '\n```\n']
stop_token_ids = [tokenizer(x)['input_ids'] for x in stop_list]
stop_token_ids = [torch.LongTensor(x).to(device) for x in stop_token_ids]
# Define custom stopping criteria class
class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_ids in stop_token_ids:
            if torch.eq(input_ids[0][-len(stop_ids):], stop_ids).all():
                return True
        return False

In [9]:
stopping_criteria = StoppingCriteriaList([StopOnTokens()])

In [11]:
generate_text = pipeline(
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    task='text-generation',
    stopping_criteria=stopping_criteria,
    temperature=0.1,
    max_new_tokens=512,
    repetition_penalty=1.1
)

# Initialize LangChain components
llm = HuggingFacePipeline(pipeline=generate_text)

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
documents = DirectoryLoader('/content/drive/MyDrive/dataa', glob="./*.pdf", loader_cls=PyPDFLoader).load()
all_splits = text_splitter.split_documents(documents)

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)
vectorstore = FAISS.from_documents(all_splits, embeddings)

chain = ConversationalRetrievalChain.from_llm(llm, vectorstore.as_retriever(), return_source_documents=True)
chat_history = []

# Define the Gradio interface function
def answer_question(query):
    result = chain({"question": query, "chat_history": chat_history})
    chat_history.append((query, result["answer"]))
    return result['answer']

<ipython-input-11-737cf4bab67c>:21: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
print(answer_question("Remedies for PCOD"))

In [14]:
import sqlite3
from datetime import datetime
import gradio as gr
import pandas as pd
import ast

# Function to safely parse the specialization column
def parse_specialization(specialization_str):
    try:
        # Use ast.literal_eval for safe parsing
        return ast.literal_eval(specialization_str)
    except (ValueError, SyntaxError):
        # Fallback: assume the string is poorly formatted, split manually
        return [s.strip() for s in specialization_str.strip("[]").split(",")]

# Load doctor data and preprocess the specialization column
doctor_data = pd.read_csv("final_cleaned.csv")
doctor_data["Specialization"] = doctor_data["Specialization"].apply(parse_specialization)

# Define the recommend_doctor function
def recommend_doctor(query):
    # Extract specialization and location from the query
    specialization = None
    location = None

    # Check for common specializations in the query
    specializations = doctor_data["Specialization"].explode().unique()
    for spec in specializations:
        if spec.lower() in query.lower():
            specialization = spec
            break

    # Check for location keywords in the query
    locations = doctor_data["Location"].unique()
    for loc in locations:
        if loc.lower() in query.lower():
            location = loc
            break

    # Filter the doctor data
    filtered_data = doctor_data.copy()
    if specialization:
        filtered_data = filtered_data[
            doctor_data["Specialization"].apply(lambda x: specialization in x)
        ]
    if location:
        filtered_data = filtered_data[filtered_data["Location"].str.lower() == location.lower()]

    # Return recommendations
    if filtered_data.empty:
        return "Sorry, no doctor found matching your query."

    recommendations = ""
    for _, row in filtered_data.iterrows():
        recommendations += f"- **Name**: {row['Name']}\n"
        recommendations += f"  **Specialization**: {', '.join(row['Specialization'])}\n"
        recommendations += f"  **Experience**: {row['Experience']} years\n"
        recommendations += f"  **Location**: {row['Location']}\n"
        recommendations += f"  **Contact**: {row['Contact Information']}\n"
        recommendations += f"  **Availability**: {row['Availability']}\n"
        recommendations += f"  **Consultation Fee**: {row['Consultation Fee']}\n\n"

    return recommendations

# Modify the format_chat_history function
def format_chat_history():
    # Connect to the SQLite database
    conn = sqlite3.connect("chat_history.db")
    cursor = conn.cursor()

    # Ensure the chat_history table exists
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS chat_history (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            question TEXT,
            answer TEXT,
            timestamp TEXT
        )
    """)

    # Retrieve the last 10 chat records
    cursor.execute("SELECT question, answer, timestamp FROM chat_history ORDER BY id DESC LIMIT 10")
    rows = cursor.fetchall()

    # Close the connection
    conn.close()

    # Format the chat history
    formatted_history = ""
    for row in rows:
        question, answer, timestamp = row
        formatted_history += f"**{timestamp}**\n"
        formatted_history += f"**Q**: {question}\n"
        formatted_history += f"**A**: {answer}\n\n"

    return formatted_history or "No chat history available."

# Modify the answer generation function
def answer_question(query):
    # Establish a persistent connection to a file-based SQLite database
    conn = sqlite3.connect("chat_history.db")
    cursor = conn.cursor()

    # Ensure the table exists
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS chat_history (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            question TEXT,
            answer TEXT,
            timestamp TEXT
        )
    """)

    # Determine if the query is about doctor recommendations
    if "doctor" in query.lower() or "recommend" in query.lower():
        answer = recommend_doctor(query)
    else:
        # Fallback to your chatbot logic
        result = chain({"question": query, "chat_history": []})
        answer = result["answer"]

    # Insert question and answer into SQLite
    cursor.execute("INSERT INTO chat_history (question, answer, timestamp) VALUES (?, ?, ?)",
                   (query, answer, str(datetime.now())))
    conn.commit()

    # Close the connection
    conn.close()

    return answer

# Gradio interface function
def chatbot_interface(user_query):
    response = answer_question(user_query)
    formatted_history = format_chat_history()
    return response, gr.update(value=formatted_history)

# Gradio setup
with gr.Blocks() as demo:
    gr.Markdown("# HerbAI 🌿 - Your Ayurveda Assistant")

    with gr.Row():
        user_input = gr.Textbox(placeholder="Ask a question about Ayurveda or doctors...", label="Your Question")
        output = gr.Textbox(label="Answer")

    history_output = gr.Textbox(label="Chat History", value=format_chat_history(), interactive=False)

    # Link the submit function
    user_input.submit(chatbot_interface, user_input, [output, history_output])

demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://0f652914bafbb9985b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://0f652914bafbb9985b.gradio.live


In [ ]:
#import time
#import gradio as gr

# Define your answer generation function
#def slow_echo(question, history=[]):
    #history = history or []
    #for i in range(len(question)):
        #ans = answer_question(question)
        #response = "Answer: " + ans
        #history.append((question, response))
        #yield history  # Yield the history for continuous updates

# Customize title, description, and examples
#title = "HerbAI 🌿 - Your Ayurveda Assistant"
#description = """
#Welcome to HerbAI🌿! I'm here to assist with your Ayurvedic health questions.
#Feel free to ask about remedies, health practices, or doshic types.
#"""

# examples = [
#    "What should we eat in order to overcome constipation?",
#    "What are the steps to determine one's prakruti and vikruti according to Ayurveda?",
#    "What are the characteristics of each doshic type in Ayurveda?",
#    "Explain breathing techniques in Ayurveda"
#]

# Setting up the Chat Interface
#demo = gr.ChatInterface(
#    fn=slow_echo,
 #   title=title,
#    description=description,
#    examples=examples,
#    theme="default",
#).queue()

# Launch the interface
#if __name__ == "__main__":
#    demo.launch(share=True,debug=True)